In [1]:
import pickle
import pandas as pd
import json

df = pd.read_csv('query_results_df.csv')

In [2]:
fixed_info = df["Info"].apply(
    lambda x: json.loads(x.replace("'", '"')) if isinstance(x, str) else x
)

df["spotify_id"] = fixed_info.apply(lambda x: x.get("id") if isinstance(x, dict) else None)

In [3]:
print(df["spotify_id"])

0     6a0uCh0i9ICDwLgjuZwhTI
1     3qLZnIVAiGexuNuG8Aptrq
2     3BYhyLzNIvSaFY2jGomMKr
3     6O7iXmIucRXi1ZmCS9jaat
4     0FvSR6SVzJRFXlneneRpcB
               ...          
95    4ApxzbzLwKXjpBBdCgfpuT
96    4SLsOKz5uYIOBHrABNs3pd
97    5d4zh95X6UeB69hZEoVz5v
98    3K42v1U1TxtjXKpJv4zy7R
99    6FwwzvOfCSLSDAJGojGWYW
Name: spotify_id, Length: 100, dtype: object


In [5]:
import requests
import pandas as pd

# Your API key
API_KEY = "aca024cc-608f-49e2-a173-1532e991d482"

# Base URL for the API
BASE_URL = "https://api.songstats.com/enterprise/v1/tracks/stats"

# Define headers
headers = {
    "apikey": API_KEY,
    "Accept-Encoding": "gzip",
}

# Function to query the API for Spotify stats
def fetch_spotify_stats(spotify_track_id):
    params = {
        "spotify_track_id": spotify_track_id,
        "source": "spotify",  # Filter for Spotify data only
        "with_charts": False,  # Exclude charts
        "with_playlists": False,  # Exclude playlists
        "with_links": False,  # Exclude source links
        "with_stations": False,  # Exclude radio stations
        "with_videos": False,  # Exclude videos
    }
    response = requests.get(BASE_URL, headers=headers, params=params)
    
    # Raise an error if the response is not successful
    response.raise_for_status()
    
    data = response.json()
    
    # Extract the stats array
    if "stats" in data and data["stats"]:
        return data["stats"][0]["data"]  # Return the first object in the stats array
    else:
        return None

# Columns to extract frosm the API
columns_to_add = [
    "streams_total", 
    "popularity_current", 
    "playlists_current", 
    "playlists_total", 
    "playlists_editorial_current", 
    "playlists_editorial_total", 
    "playlist_reach_current", 
    "playlist_reach_total", 
    "charts_current", 
    "charts_total",
]

# Add the columns with default values (e.g., None)
for col in columns_to_add:
    df[col] = pd.NA

# Iterate through the DataFrame and fetch stats for each Spotify ID
# Iterate through the DataFrame and fetch stats for each Spotify ID
for index, row in df.iterrows():
    try:
        stats_data = fetch_spotify_stats(row["spotify_id"])
        if stats_data:
            for col in columns_to_add:
                df.loc[index, col] = stats_data.get(col, pd.NA)  # Use the column name directly
    except Exception as e:
        print(f"Error fetching data for Spotify ID {row['spotify_id']}: {e}")

print(df.head())
# Save the updated DataFrame to a new CSV
df.to_csv("songstats_data_included_final.csv", index=False)
print("Updated DataFrame saved to 'songstats_data_included.csv'.")


Error fetching data for Spotify ID 7jQS5Pl3fGAL4Lxdfg99VF: 404 Client Error: Not Found for url: https://api.songstats.com/enterprise/v1/tracks/stats?spotify_track_id=7jQS5Pl3fGAL4Lxdfg99VF&source=spotify&with_charts=False&with_playlists=False&with_links=False&with_stations=False&with_videos=False
Error fetching data for Spotify ID 6vgdTVpCtm59R3fOgdyYHv: 404 Client Error: Not Found for url: https://api.songstats.com/enterprise/v1/tracks/stats?spotify_track_id=6vgdTVpCtm59R3fOgdyYHv&source=spotify&with_charts=False&with_playlists=False&with_links=False&with_stations=False&with_videos=False
Error fetching data for Spotify ID 0iQJMd2BZlY9WnUxglCYgG: 404 Client Error: Not Found for url: https://api.songstats.com/enterprise/v1/tracks/stats?spotify_track_id=0iQJMd2BZlY9WnUxglCYgG&source=spotify&with_charts=False&with_playlists=False&with_links=False&with_stations=False&with_videos=False
Error fetching data for Spotify ID 6zUOshjRmV5029qYbfE3q5: 404 Client Error: Not Found for url: https://ap